In [ ]:
import numpy as np
import os
import sys
sys.path.append(r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project')

import matplotlib.pyplot as plt # for plotting
%matplotlib widget
import pandas as pd # for data manipulation
from Models.load_data import *
from Models.helpers import *

%load_ext autoreload
%autoreload 2

import torch

path = r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Training_data'
features_list = os.listdir(os.path.join(path, r'features'))

In [ ]:
# This cell takes 19s
# Instantiate the dataset without transformations
dataset = IndependentCSVDataset(path, features_list)

# Split dataset into training and testing sets
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

feature_min, feature_max = compute_global_minmax(train_dataset)
print("Global feature min (train set only):", feature_min)
print("Global feature max (train set only):", feature_max)

global_transform = GlobalMinMaxNormalize(feature_min, feature_max)
train_dataset.dataset.transform = global_transform
test_dataset.dataset.transform = global_transform  # Same transform to prevent data leakage

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)



In [ ]:
# Define model parameters
input_size = 6       # Number of input features per time step
hidden_size = 32     # Transformer hidden size
num_layers = 2       # Number of Transformer encoder layers
output_size = 1      # Binary classification
dropout = 0.2
num_heads = 4        # Number of attention heads

# Instantiate the Transformer model
model = TransformerModel(input_size, hidden_size, num_layers, output_size, dropout, num_heads)

# Print model architecture
print(model)

# Example input: (batch_size=2, sequence_length=6000, input_size=6)
x_sample = torch.randn(2, 6000, 6)

# Predict with threshold 0.5
predictions, probabilities = model.predict(x_sample, threshold=0.5)

print("Predictions:\n", predictions)
print("Probabilities:\n", probabilities)


In [ ]:
# With 10 rounds of optimisation, this takes roughly 30 min on my laptop
import torch
import torch.optim as optim

# Define loss and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss(pos_weight=compute_class_weights(train_loader))
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Assume `train_loader` is a DataLoader yielding batches of (input, target)
num_epochs = 2

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        print(f'Accumulated epoch loss: {epoch_loss-1:.4f}', end='\r')
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")
